<a href="https://colab.research.google.com/github/amey-joshi/am/blob/master/optim/Multi_objective_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
import numpy as np
import pandas as pd
import cvxpy as cp

from sklearn.preprocessing import StandardScaler
from scipy import optimize

# Generate simulated data

We simulate data assuming that all the variables are normally distributed. The mean and the standard deviations of the variables are assumed to take the values in the variables <code>xmeans</code> and <code>xstds</code>.

In [131]:
n_obs = 1160 # Number of observations
xmeans = [0.15, 130.84, 85.73, 1046.68, 320.59]
xstds  = [0.014, 14.29, 8.79, 114.32, 25.43]

np.random.seed(11112020)
x1 = np.random.normal(xmeans[0], xstds[0], n_obs)
x2 = np.random.normal(xmeans[1], xstds[1], n_obs)
x3 = np.random.normal(xmeans[2], xstds[2], n_obs)
x4 = np.random.normal(xmeans[3], xstds[3], n_obs)
x5 = np.random.normal(xmeans[4], xstds[4], n_obs)

ymeans = [282.38, 107.12, 0.01, 0.38, 1.31]
ystds  = [27.21, 23.57, 0.006, 0.07, 0.09]

y1 = np.random.normal(ymeans[0], ystds[0], n_obs)
y2 = np.random.normal(ymeans[1], ystds[1], n_obs)
y3 = np.random.normal(ymeans[2], ystds[2], n_obs)
y4 = np.random.normal(ymeans[3], ystds[3], n_obs)
y5 = np.random.normal(ymeans[4], ystds[4], n_obs)

data = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5,
                     'y1': y1, 'y2': y2, 'y3': y3, 'y4': y4, 'y5': y5,})

In the sections to follow, we will predict $y_i$ as functions of $x_1, \ldots, x_5$ assuming a variety of functional forms of the dependency. We will start with the linear function and then try a few non-linear ones.

In [132]:
data.head()

,x1,x2,x3,x4,x5,y1,y2,y3,y4,y5
0,0.147559,133.622875,94.997665,1019.485754,326.688804,281.546963,121.427647,0.007834,0.381522,1.327320
1,0.167399,162.109384,96.851700,1182.521642,289.111681,252.290334,107.480755,0.014776,0.374861,1.336146
2,0.139575,132.074400,94.824109,905.522131,342.808755,299.148819,99.305775,0.003770,0.353045,1.345665
3,0.160830,121.043438,73.329479,1090.703080,334.113073,256.046095,112.571144,0.012496,0.458908,1.208326
4,0.164814,147.996518,84.611238,1251.623422,304.861298,208.441233,87.748032,0.012011,0.429719,1.302457


# Common functions

In [133]:
def compute_error_orig(X, Y, beta, scaler, X1 = None):
  """ Compute the error between unscaled variables. 
  
  Some times, we run a model on scaled variables. In that case, we apply the
  inverse transform on the results and compare them with the original data.
  """
  if X1 is None:
    Y_hat = cp.matmul(X, beta)
  else:
    Y_hat = cp.matmul(X1, beta)

  Z = np.concatenate((X, Y_hat.value), axis=1)
  results = pd.DataFrame(data = scaler.inverse_transform(Z))
  results.columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'y1', 'y2', 'y3', 'y4', 'y5']
  error = (data - results)**2
  total_errors = error.sum().to_numpy()[5:]
  rms_error = np.sqrt(total_errors)/n_obs
  print(rms_error)

  return rms_error

In [134]:
def get_unscaled_yhat(X, Y, beta, scaler, X1 = None):
  if X1 is None:
    Y_hat = cp.matmul(X, beta)
  else:
    Y_hat = cp.matmul(X1, beta)
    
  Z = np.concatenate((X, Y_hat.value), axis=1)
  results = pd.DataFrame(data = scaler.inverse_transform(Z))
  results.columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'y1', 'y2', 'y3', 'y4', 'y5']

  return results[['y1', 'y2', 'y3', 'y4', 'y5']]

In [135]:
def compute_error(X, Y, beta):
  """
  Compute error between predicted and actual response. 
  """
  Y_hat = cp.matmul(X, beta)
  error = Y - Y_hat
  error_sq = np.square(error.value)
  sse = np.sum(error_sq, axis=0)
  rms = np.sqrt(sse)/n_obs
  print(f'Total sum of squares of error: {sse}')
  print(f'Root mean square of error: {rms}')

  return rms

In [136]:
def print_status(prob, beta):
  """ Prints the status of the optimizer. """
  print(f'Problem status: {prob.status}')
  print(f'Optimal value: {prob.value}')
  print('Beta values:')
  print(beta.value)

This data frame will store the RMS errors of each model.

In [137]:
column_names = ['model_name', 'err_y1', 'err_y2', 'err_y3', 'err_y4', 'err_y5']
model_summary = pd.DataFrame(columns=column_names)

# Linear model - 1

We are fitting the model:

\begin{eqnarray}
y_1 &=& \beta_{11}x_1 + \beta_{12}x_2 + \beta_{13}x_3 + \beta_{14}x_4 + \beta_{15}x_5 \\
y_2 &=& \beta_{21}x_1 + \beta_{22}x_2 + \beta_{23}x_3 + \beta_{24}x_4 + \beta_{25}x_5 \\
y_3 &=& \beta_{31}x_1 + \beta_{32}x_2 + \beta_{33}x_3 + \beta_{34}x_4 + \beta_{35}x_5 \\
y_4 &=& \beta_{41}x_1 + \beta_{42}x_2 + \beta_{43}x_3 + \beta_{44}x_4 + \beta_{45}x_5 \\
y_5 &=& \beta_{51}x_1 + \beta_{52}x_2 + \beta_{53}x_3 + \beta_{54}x_4 + \beta_{55}x_5
\end{eqnarray}

Optimizers work best when the variables are scaled. We use the scikit-learn scaler to transform our data. 

In [138]:
scaler = StandardScaler()
scaler.fit(data)
scaled_data = scaler.transform(data)

We segregate the preditors into the matrix <code>X</code> and the responses into the matrix <code>Y</code>.

In [139]:
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]

We now build a linear model and run it. The variable <code>beta</code> has the coeffifients of the linear model.

**This step can take a few minutes.**

In [140]:
beta = cp.Variable((5, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X, beta)))
prob = cp.Problem(obj)
prob.solve()

35.5324071011577

In [141]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 35.5324071011577
Beta values:
[[ 0.0109488   0.02589171 -0.0442837   0.02470366 -0.02229602]
 [ 0.00994741 -0.03560378  0.02444012  0.01066956  0.0111661 ]
 [-0.0201376   0.02608267  0.02757303 -0.00720288  0.03756629]
 [ 0.03318335 -0.01143796 -0.01100121  0.02615937 -0.05206846]
 [ 0.03577049 -0.04468985  0.01498306 -0.02103517 -0.02175626]]


In [142]:
compute_error(X, Y, beta)

Total sum of squares of error: [1156.61287274 1154.53861065 1155.74789072 1157.75610247 1154.04109214]
Root mean square of error: [0.02931811 0.02929181 0.02930715 0.0293326  0.0292855 ]


array([0.02931811, 0.02929181, 0.02930715, 0.0293326 , 0.0292855 ])

In [143]:
errors = compute_error_orig(X, Y, beta, scaler)

[7.88313341e-01 6.92319500e-01 1.75097276e-04 2.01302848e-03
 2.63304027e-03]


In [144]:
y_hat_1 = get_unscaled_yhat(X, Y, beta, scaler)

In [145]:
print(y_hat_1[0:10]) # printing just a sample.

           y1          y2        y3        y4        y5
0  281.647849  106.400085  0.010226  0.380068  1.317398
1  282.359469  106.796852  0.009970  0.388451  1.313624
2  281.184061  105.748332  0.010475  0.376385  1.321727
3  283.814755  105.774883  0.009432  0.382873  1.302990
4  283.888586  105.898824  0.009658  0.388265  1.304366
5  284.577330  104.384273  0.009725  0.382892  1.304497
6  283.447690  105.120773  0.009710  0.383979  1.304408
7  281.725487  106.984351  0.009871  0.381486  1.312981
8  281.243676  107.309448  0.010033  0.380090  1.316658
9  282.300454  106.220300  0.010221  0.379268  1.314925


In [146]:
model_summary = model_summary.append({'model_name': 'Linear model - 1', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Linear model - 2
We will fit a linear model with an intercept.

\begin{eqnarray}
y_1 &=& \beta_{11}x_1 + \beta_{12}x_2 + \beta_{13}x_3 + \beta_{14}x_4 + \beta_{15}x_5 + \beta_{10}\\
y_2 &=& \beta_{21}x_1 + \beta_{22}x_2 + \beta_{23}x_3 + \beta_{24}x_4 + \beta_{25}x_5 + \beta_{20} \\
y_3 &=& \beta_{31}x_1 + \beta_{32}x_2 + \beta_{33}x_3 + \beta_{34}x_4 + \beta_{35}x_5 + \beta_{30} \\
y_4 &=& \beta_{41}x_1 + \beta_{42}x_2 + \beta_{43}x_3 + \beta_{44}x_4 + \beta_{45}x_5 + \beta_{40} \\
y_5 &=& \beta_{51}x_1 + \beta_{52}x_2 + \beta_{53}x_3 + \beta_{54}x_4 + \beta_{55}x_5 + \beta_{50}
\end{eqnarray}

In [147]:
X1 = np.hstack((X, np.ones((n_obs, 1))))
beta = cp.Variable((6, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X1, beta)))
prob = cp.Problem(obj)
prob.solve()

35.535277027584655

In [148]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 35.535277027584655
Beta values:
[[ 1.09397889e-02  2.59220080e-02 -4.43067351e-02  2.48319614e-02
  -2.21265280e-02]
 [ 1.00519081e-02 -3.56046136e-02  2.47613889e-02  1.09339153e-02
   1.12205095e-02]
 [-2.02438936e-02  2.62953456e-02  2.76433063e-02 -7.35515989e-03
   3.72673410e-02]
 [ 3.30284669e-02 -1.12285498e-02 -1.08815323e-02  2.60963434e-02
  -5.23252671e-02]
 [ 3.59263790e-02 -4.47787525e-02  1.50764993e-02 -2.11889100e-02
  -2.19615602e-02]
 [ 9.43465094e-05  6.67892246e-06  1.90589107e-04  1.46015662e-04
   3.20768804e-05]]


In [149]:
compute_error(X1, Y, beta)

Total sum of squares of error: [1156.61297354 1154.53873561 1155.74810773 1157.75627625 1154.04137772]
Root mean square of error: [0.02931811 0.02929181 0.02930715 0.0293326  0.0292855 ]


array([0.02931811, 0.02929181, 0.02930715, 0.0293326 , 0.0292855 ])

In [150]:
errors = compute_error_orig(X, Y, beta, scaler, X1)

[7.88313375e-01 6.92319537e-01 1.75097293e-04 2.01302863e-03
 2.63304059e-03]


In [151]:
y_hat_2 = get_unscaled_yhat(X, Y, beta, scaler, X1)

In [152]:
print(y_hat_2[0:10]) # print a sample of estimated Y.

           y1          y2        y3        y4        y5
0  281.649900  106.403730  0.010228  0.380068  1.317373
1  282.354470  106.812487  0.009976  0.388508  1.313620
2  281.192676  105.745318  0.010477  0.376376  1.321705
3  283.819599  105.769458  0.009431  0.382885  1.303020
4  283.884741  105.909133  0.009662  0.388307  1.304365
5  284.587298  104.380090  0.009728  0.382917  1.304511
6  283.448762  105.119022  0.009711  0.384002  1.304430
7  281.724636  106.985929  0.009871  0.381488  1.312983
8  281.243221  107.312474  0.010033  0.380083  1.316643
9  282.301881  106.225362  0.010222  0.379249  1.314873


In [153]:
model_summary = model_summary.append({'model_name': 'Linear model - 2', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Nonlinear model - 1

We fit the model:

\begin{eqnarray}
\ln y_1 &=& \beta_{11}x_1 + \beta_{12}\ln x_2 + \beta_{13}\ln x_3 + \beta_{14}\ln x_4 + \beta_{15}\ln x_5 \\
\ln y_2 &=& \beta_{21}x_1 + \beta_{22}\ln x_2 + \beta_{23}\ln x_3 + \beta_{24}\ln x_4 + \beta_{25}\ln x_5 \\
y_3 &=& \beta_{31}x_1 + \beta_{32}\ln x_2 + \beta_{33}\ln x_3 + \beta_{34}\ln x_4 + \beta_{35}\ln x_5 \\
y_4 &=& \beta_{41}x_1 + \beta_{42}\ln x_2 + \beta_{43}\ln x_3 + \beta_{44}\ln x_4 + \beta_{45}\ln x_5 \\
y_5 &=& \beta_{51}x_1 + \beta_{52}\ln x_2 + \beta_{53}\ln x_3 + \beta_{54}\ln x_4 + \beta_{55}\ln x_5 \\
\end{eqnarray}

In [154]:
data_nm1 = data.copy()

data_nm1['x2'] = np.log(data_nm1['x2'])
data_nm1['x3'] = np.log(data_nm1['x3'])
data_nm1['x4'] = np.log(data_nm1['x4'])
data_nm1['x5'] = np.log(data_nm1['x5'])

data_nm1['y1'] = np.log(data_nm1['y1'])
data_nm1['y2'] = np.log(data_nm1['y2'])

**This step can take a long time.**

In [155]:
X = data_nm1[['x1', 'x2', 'x3', 'x4', 'x5']].to_numpy(copy=True)
Y = data_nm1[['y1', 'y2', 'y3', 'y4', 'y5']].to_numpy(copy=True)
beta = cp.Variable((5, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X, beta)))
prob = cp.Problem(obj)
prob.solve()

8.305100187417185

In [156]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 8.305100187417185
Beta values:
[[ 4.24407642e-01  9.19228230e-01 -1.83308583e-02  1.39424365e-01
  -4.27318804e-02]
 [ 1.79095648e-01  8.50325176e-02  1.39617787e-03  1.83790729e-02
   5.97255814e-02]
 [ 1.32647535e-01  2.30710221e-01  1.33360064e-03  6.37619860e-03
   7.65496021e-02]
 [ 2.69589815e-01  2.45463165e-01 -7.88288533e-04  3.26207990e-02
   3.19679261e-02]
 [ 3.88274684e-01  2.34984382e-01  9.42597893e-04  2.78823336e-03
   8.07657252e-02]]


In [157]:
errors = compute_error_orig(X, Y, beta, scaler)

[4.52004980e+00 3.29259774e+00 1.75424015e-04 2.15626246e-03
 4.35624627e-03]


In [158]:
y_hat_3 = get_unscaled_yhat(X, Y, beta, scaler)

In [159]:
print(y_hat_3[0:10]) # print a sample of estimated Y.

           y1          y2        y3        y4        y5
0  434.340306  216.398206  0.010015  0.407615  1.431778
1  435.365908  217.505051  0.010013  0.408366  1.432412
2  433.830094  215.771112  0.010017  0.407267  1.431742
3  433.816325  215.592680  0.010011  0.407659  1.429771
4  435.381340  217.153151  0.010012  0.408304  1.431551
5  435.275632  216.551782  0.010012  0.407896  1.431321
6  433.694468  215.296687  0.010012  0.407734  1.429839
7  433.415170  215.700660  0.010013  0.407553  1.430478
8  433.601798  215.994919  0.010014  0.407497  1.431018
9  434.956868  216.913743  0.010015  0.407652  1.432120


In [160]:
model_summary = model_summary.append({'model_name': 'Nonlinear model - 1', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Nonlinear model - 1a

We fit the model:

\begin{eqnarray}
\ln y_1 &=& \beta_{11}x_1 + \beta_{12}\ln x_2 + \beta_{13}\ln x_3 + \beta_{14}\ln x_4 + \beta_{15}\ln x_5 + \beta_{10}\\
\ln y_2 &=& \beta_{21}x_1 + \beta_{22}\ln x_2 + \beta_{23}\ln x_3 + \beta_{24}\ln x_4 + \beta_{25}\ln x_5 + \beta_{20} \\
y_3 &=& \beta_{31}x_1 + \beta_{32}\ln x_2 + \beta_{33}\ln x_3 + \beta_{34}\ln x_4 + \beta_{35}\ln x_5 + \beta_{30} \\
y_4 &=& \beta_{41}x_1 + \beta_{42}\ln x_2 + \beta_{43}\ln x_3 + \beta_{44}\ln x_4 + \beta_{45}\ln x_5 + \beta_{40} \\
y_5 &=& \beta_{51}x_1 + \beta_{52}\ln x_2 + \beta_{53}\ln x_3 + \beta_{54}\ln x_4 + \beta_{55}\ln x_5 + \beta_{50} \\
\end{eqnarray}

In [161]:
data_nm1 = data.copy()

data_nm1['x2'] = np.log(data_nm1['x2'])
data_nm1['x3'] = np.log(data_nm1['x3'])
data_nm1['x4'] = np.log(data_nm1['x4'])
data_nm1['x5'] = np.log(data_nm1['x5'])

data_nm1['y1'] = np.log(data_nm1['y1'])
data_nm1['y2'] = np.log(data_nm1['y2'])

X = data_nm1[['x1', 'x2', 'x3', 'x4', 'x5']].to_numpy(copy=True)
Y = data_nm1[['y1', 'y2', 'y3', 'y4', 'y5']].to_numpy(copy=True)
X1 = np.hstack((X, np.ones((n_obs, 1))))
beta = cp.Variable((6, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X1, beta)))
prob = cp.Problem(obj)
prob.solve()

8.131718387558735

In [162]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 8.131718387558735
Beta values:
[[ 8.90788558e-02  5.74988290e-01 -9.50026868e-03  1.16636348e-01
  -1.42652441e-01]
 [ 1.13444977e-02 -8.72357366e-02  1.85781502e-03  7.74110256e-03
   8.80722709e-03]
 [-2.17434418e-02  7.21693077e-02 -5.44201275e-05 -3.12573944e-03
   2.94501552e-02]
 [ 2.82224982e-02 -2.56561620e-03 -2.14124802e-03  1.77753164e-02
  -4.20399104e-02]
 [ 4.09692015e-02 -1.22079097e-01  8.09385983e-04 -1.89398338e-02
  -2.47706318e-02]
 [ 5.23364643e+00  5.37800571e+00  1.26805378e-02  3.25917593e-01
   1.59519489e+00]]


In [163]:
errors = compute_error_orig(X, Y, beta, scaler, X1)

[4.52037271e+00 3.29246071e+00 1.75425132e-04 2.15629645e-03
 4.35568142e-03]


In [164]:
y_hat_4 = get_unscaled_yhat(X, Y, beta, scaler, X1)

In [165]:
print(y_hat_4[0:10]) # Print a sample of estimates of y

           y1          y2        y3        y4        y5
0  433.733381  215.850560  0.010014  0.407518  1.431159
1  433.806517  216.098289  0.010012  0.408115  1.430820
2  433.674873  215.631182  0.010016  0.407241  1.431588
3  433.962362  215.724240  0.010011  0.407681  1.429919
4  433.953064  215.863988  0.010011  0.408075  1.430090
5  434.030702  215.427267  0.010013  0.407694  1.430056
6  433.926865  215.506600  0.010012  0.407771  1.430068
7  433.742817  215.996932  0.010012  0.407606  1.430808
8  433.691470  216.076336  0.010013  0.407512  1.431108
9  433.787656  215.857915  0.010013  0.407466  1.430927


In [166]:
model_summary = model_summary.append({'model_name': 'Nonlinear model - 1a', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Nonlinear model - 2

Transform the variables. We are fitting the model:

\begin{eqnarray}
y_1 &=& \beta_{11}x_1 + \beta_{12}\ln x_2 + \beta_{13}\ln x_3 + \beta_{14}\ln x_4 + \beta_{15}\ln x_5 \\
y_2 &=& \beta_{21}x_1 + \beta_{22}\ln x_2 + \beta_{23}\ln x_3 + \beta_{24}\ln x_4 + \beta_{25}\ln x_5 \\
y_3 &=& \beta_{31}x_1 + \beta_{32}\ln x_2 + \beta_{33}\ln x_3 + \beta_{34}\ln x_4 + \beta_{35}\ln x_5 \\
y_4 &=& \beta_{41}x_1 + \beta_{42}\ln x_2 + \beta_{43}\ln x_3 + \beta_{44}\ln x_4 + \beta_{45}\ln x_5 \\
y_5 &=& \beta_{51}x_1 + \beta_{52}\ln x_2 + \beta_{53}\ln x_3 + \beta_{54}\ln x_4 + \beta_{55}\ln x_5 \\
\end{eqnarray}

In [167]:
data_nm2 = data.copy()

data_nm2['x2'] = np.log(data_nm2['x2'])
data_nm2['x3'] = np.log(data_nm2['x3'])
data_nm2['x4'] = np.log(data_nm2['x4'])
data_nm2['x5'] = np.log(data_nm2['x5'])

**This step can take a few minutes.**

In [168]:
X = data_nm2[['x1', 'x2', 'x3', 'x4', 'x5']].to_numpy(copy=True)
Y = data_nm2[['y1', 'y2', 'y3', 'y4', 'y5']].to_numpy(copy=True)
beta = cp.Variable((5, 5))
obj = cp.Minimize(cp.norm(Y - cp.matmul(X, beta)))
prob = cp.Problem(obj)
prob.solve()

918.4829557994454

In [169]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 918.4829557994454
Beta values:
[[ 3.25715798e+01  5.62077151e+01 -3.60577321e-01 -5.80506727e-01
  -1.25713030e+00]
 [ 8.51565325e+00 -2.20178778e+00 -1.87710797e-01 -1.13926160e-01
  -5.18954634e-03]
 [-1.26475186e+00  1.18470501e+01  1.30673201e-01  4.18812154e-02
   2.99929555e-01]
 [ 1.55160765e+01  7.75954533e+00  1.75203901e-01  5.82633512e-02
  -1.17186472e-01]
 [ 2.31772235e+01  3.30955793e-01 -1.41628782e-01  7.27569643e-02
   1.71138948e-01]]


In [170]:
errors = compute_error_orig(X, Y, beta, scaler)

[2.22786559e+02 7.36892307e+01 1.75582977e-04 2.14874247e-03
 4.31356397e-03]


In [171]:
y_hat_5 = get_unscaled_yhat(X, Y, beta, scaler)

In [172]:
print(y_hat_5[0:10]) # print a sample of estimates.

            y1           y2        y3        y4        y5
0  7875.032281  2638.288941  0.010054  0.406962  1.432700
1  7921.740174  2686.253305  0.010069  0.404700  1.427445
2  7845.993239  2606.412784  0.009918  0.407133  1.435549
3  7914.995404  2601.133040  0.009986  0.406845  1.423899
4  7959.965942  2660.556233  0.010085  0.405618  1.424354
5  7978.935180  2614.911961  0.009824  0.406057  1.426029
6  7902.306809  2582.078092  0.010050  0.406723  1.423505
7  7847.204149  2623.552049  0.010139  0.407162  1.429030
8  7840.647869  2637.893333  0.010139  0.407333  1.431794
9  7908.842996  2655.002807  0.010122  0.407801  1.433688


In [173]:
model_summary = model_summary.append({'model_name': 'Nonlinear model - 2', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Linear model using Nelder-Mead algorithm

We are fitting the model:

\begin{eqnarray}
y_1 &=& \beta_{11}x_1 + \beta_{12}x_2 + \beta_{13}x_3 + \beta_{14}x_4 + \beta_{15}x_5 \\
y_2 &=& \beta_{21}x_1 + \beta_{22}x_2 + \beta_{23}x_3 + \beta_{24}x_4 + \beta_{25}x_5 \\
y_3 &=& \beta_{31}x_1 + \beta_{32}x_2 + \beta_{33}x_3 + \beta_{34}x_4 + \beta_{35}x_5 \\
y_4 &=& \beta_{41}x_1 + \beta_{42}x_2 + \beta_{43}x_3 + \beta_{44}x_4 + \beta_{45}x_5 \\
y_5 &=& \beta_{51}x_1 + \beta_{52}x_2 + \beta_{53}x_3 + \beta_{54}x_4 + \beta_{55}x_5
\end{eqnarray}

In [174]:
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]

def error_fn0(beta):
  return np.linalg.norm(Y[:, 0] - X @ beta)

def error_fn1(beta):
  return np.linalg.norm(Y[:, 1] - X @ beta)

def error_fn2(beta):
  return np.linalg.norm(Y[:, 2] - X @ beta)

def error_fn3(beta):
  return np.linalg.norm(Y[:, 3] - X @ beta)

def error_fn4(beta):
  return np.linalg.norm(Y[:, 4] - X @ beta)

beta = np.zeros((5, 5))

def run_nelder_mead_solver(beta):
  r0 = optimize.minimize(error_fn0, beta[0, :], method='nelder-mead')
  r1 = optimize.minimize(error_fn1, beta[1, :], method='nelder-mead')
  r2 = optimize.minimize(error_fn2, beta[2, :], method='nelder-mead')
  r3 = optimize.minimize(error_fn3, beta[3, :], method='nelder-mead')
  r4 = optimize.minimize(error_fn4, beta[4, :], method='nelder-mead')

  beta[0, :] = r0.fun
  beta[1, :] = r1.fun
  beta[2, :] = r2.fun
  beta[3, :] = r3.fun
  beta[4, :] = r4.fun

  return beta

beta = run_nelder_mead_solver(beta)


In [175]:
errors = compute_error_orig(X, Y, beta, scaler)

[6.07828666e+01 5.34628055e+01 1.35100176e-02 1.55170923e-01
 2.03375578e-01]


In [176]:
y_hat_6 = get_unscaled_yhat(X, Y, beta, scaler)

In [177]:
print(y_hat_6[0:10])

            y1           y2        y3         y4         y5
0  1266.572904   971.471389  0.228683   2.893885   4.604357
1  4579.689845  3883.758801  0.964856  11.350046  15.682800
2   301.324289   123.001069  0.014205   0.430253   1.376746
3  -105.352502  -234.474903 -0.076158  -0.607719   0.016894
4  3326.163667  2781.887489  0.686323   8.150636  11.491243
5  2375.672409  1946.389143  0.475124   5.724670   8.312978
6  -835.564224  -876.343707 -0.238411  -2.471459  -2.424797
7  -419.444425  -510.567167 -0.145949  -1.409385  -1.033370
8   177.907231    14.515328 -0.013218   0.115252   0.964062
9  1964.020792  1584.540212  0.383656   4.674000   6.936492


In [178]:
model_summary = model_summary.append({'model_name': 'Nelder-Mead', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

# Levenberg-Marquardt algorithm

The <code>scipy</code> function <code>leastsq</code> used the Levenberg-Marquardt algorithm to find the values of the coefficients that minimize the squared error. Refer to the notes section of the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.leastsq.html#scipy.optimize.leastsq) and the MINPACK documentation of [lmder](https://www.math.utah.edu/software/minpack/minpack/lmder.html) and [lmdif](https://www.math.utah.edu/software/minpack/minpack/lmdif.html).


In [179]:
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]

def error_fn0(beta):
  return (Y[:, 0] - X @ beta)

def error_fn1(beta):
  return (Y[:, 1] - X @ beta)

def error_fn2(beta):
  return (Y[:, 2] - X @ beta)

def error_fn3(beta):
  return (Y[:, 3] - X @ beta)

def error_fn4(beta):
  return (Y[:, 4] - X @ beta)

beta = np.zeros((5, 5))

def run_lm_solver(beta):
  r0 = optimize.leastsq(error_fn0, beta[0, :])
  r1 = optimize.leastsq(error_fn1, beta[1, :])
  r2 = optimize.leastsq(error_fn2, beta[2, :])
  r3 = optimize.leastsq(error_fn3, beta[3, :])
  r4 = optimize.leastsq(error_fn4, beta[4, :])

  beta[0, :] = r0[0]
  beta[1, :] = r1[0]
  beta[2, :] = r2[0]
  beta[3, :] = r3[0]
  beta[4, :] = r4[0]

  return beta

beta = run_lm_solver(beta)

In [180]:
errors = compute_error_orig(X, Y, beta, scaler)

[7.90030729e-01 6.93673148e-01 1.75192641e-04 2.01462603e-03
 2.64331028e-03]


In [181]:
y_hat_7 = get_unscaled_yhat(X, Y, beta, scaler)

In [182]:
print(y_hat_7[0:10])

           y1          y2        y3        y4        y5
0  280.761377  106.587177  0.010232  0.378947  1.312936
1  284.159888  105.273293  0.010034  0.388038  1.309598
2  279.517117  106.454459  0.010460  0.373791  1.312196
3  283.526396  106.402410  0.009616  0.383704  1.314616
4  285.058617  105.608888  0.009782  0.388365  1.308961
5  283.457481  105.807219  0.010042  0.380224  1.309301
6  284.835954  105.339897  0.009624  0.384939  1.308141
7  281.940711  106.599228  0.009815  0.382945  1.315382
8  280.622615  107.044601  0.010008  0.380843  1.316775
9  279.948659  107.406021  0.010324  0.377461  1.313541


In [183]:
model_summary = model_summary.append({'model_name': 'Levenberg-Marquardt', 'err_y1': errors[0], 'err_y2': errors[1], 'err_y3': errors[2], 'err_y4': errors[3], 'err_y5': errors[4]}, ignore_index=True)

In [184]:
model_summary

,model_name,err_y1,err_y2,err_y3,err_y4,err_y5
0,Linear model - 1,0.788313,0.692319,0.000175,0.002013,0.002633
1,Linear model - 2,0.788313,0.692320,0.000175,0.002013,0.002633
2,Nonlinear model - 1,4.520050,3.292598,0.000175,0.002156,0.004356
3,Nonlinear model - 1a,4.520373,3.292461,0.000175,0.002156,0.004356
4,Nonlinear model - 2,222.786559,73.689231,0.000176,0.002149,0.004314
5,Nelder-Mead,60.782867,53.462805,0.013510,0.155171,0.203376
6,Levenberg-Marquardt,0.790031,0.693673,0.000175,0.002015,0.002643


# Conclusion
The model summary indicates that the non-linear model has given the best results on simulated data. The exercise needs to repeated on real data to get the best results.

# Weighted model - 1

In [185]:
weights = 0.2 * np.ones(5)
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]
y = np.average(Y, axis=1, weights=weights)
y = y.reshape((n_obs, 1))

In [186]:
beta = cp.Variable((5, 1))
obj = cp.Minimize(cp.sum_squares(y - X@beta))
prob = cp.Problem(obj)
prob.solve()

222.43462549212643

In [187]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 222.43462549212643
Beta values:
[[-0.00100429]
 [ 0.00412584]
 [ 0.01277479]
 [-0.00303544]
 [-0.00735161]]


In [188]:
Y_hat = cp.matmul(X, beta) # Estimates of Y.

In [189]:
errors = compute_error(X, y, beta)

Total sum of squares of error: [222.43462549]
Root mean square of error: [0.01285711]


In [190]:
model_summary = model_summary.append({'model_name': 'Weighted model - 1', 'err_y1': errors[0], 'err_y2': None, 'err_y3': None, 'err_y4': None, 'err_y5': None}, ignore_index=True)

# Weighted model - 1a

In [191]:
weights = np.ones(5)/5
X = scaled_data[:, 0:5]
Y = scaled_data[:, 5:10]
y = np.average(Y, axis=1, weights=weights)
y = y.reshape((n_obs, 1))

In [192]:
X1 = np.hstack((X, np.ones((n_obs, 1))))
beta = cp.Variable((6, 1))
obj = cp.Minimize(cp.sum_squares(y - X1@beta))
prob = cp.Problem(obj)
prob.solve()

222.4346254921264

In [193]:
print_status(prob, beta)

Problem status: optimal
Optimal value: 222.4346254921264
Beta values:
[[-1.00429183e-03]
 [ 4.12584027e-03]
 [ 1.27747863e-02]
 [-3.03544212e-03]
 [-7.35160715e-03]
 [ 5.09418662e-18]]


In [194]:
Y_hat = cp.matmul(X1, beta) # Estimates of Y.

In [195]:
errors = compute_error(X1, y, beta)

Total sum of squares of error: [222.43462549]
Root mean square of error: [0.01285711]


In [196]:
model_summary = model_summary.append({'model_name': 'Weighted model - 1a', 'err_y1': errors[0], 'err_y2': None, 'err_y3': None, 'err_y4': None, 'err_y5': None}, ignore_index=True)